In [7]:
import pandas as pd
from pathlib import Path
import glob

Load repository to Colab 

In [4]:
%cd /content
!rm -rf gps-lane-changes
!git clone https://github.com/tomdries/gps-lane-changes
%cd gps-lane-changes

/content
Cloning into 'gps-lane-changes'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 12 (delta 2), reused 10 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.
/content/gps-lane-changes


In [11]:
lca_dfs = []
for i in range(3):
    lca_dfs.append(pd.read_csv(f'data/lane change annotations/lane change annotations with sections trip {i+1}.csv'))